In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_tuner as kt
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#loading the trajectories with three peaks and their corresponding labels
xtrainx3_1 = np.loadtxt('../Data/Xtrainx3_1.csv', delimiter=',')
xtrainx3_2 = np.loadtxt('../Data/Xtrainx3_2.csv', delimiter=',')
#files with trajectories were split to be small enough to upload to GitHub, here we concatenate them
xtrainx3 = np.concatenate((xtrainx3_1, xtrainx3_2))
ytrain3 = np.loadtxt('../Data/Ytrain3.csv', delimiter=',')

In [ ]:
#concatenate an index column to the original data to keep track of the original row positions (trajectories). This
#is important because after operations like filtering, shuffling, and training, we may lose track of the original 
#correspondence between each trajectory and its original position in the dataset. By appending the row indices as a
#new column, we can still identify each trajectory later for further analysis.
xtrainx3_index = np.concatenate((xtrainx3, np.arange(xtrainx3.shape[0]).reshape(xtrainx3.shape[0], 1)), axis=1)

#ytrain3[:,[2,6,10]] selects the columns with the positions of the three peaks, located at indices 2, 6, and 10. 
nu3 = ytrain3[:,[2,6,10]]

In [ ]:
#function to split the data into training, validation and test sets, and calculate their Fourier coefficients along
#with the corresponding labels. The function also appends the original index of each trajectory for tracking.
def fouriertrainvaltest(X, Y, Ntrain, Nval, Ntest):
    
    #Generating a training set with Ntrain trajectories, a validation with Nval trajectories and a test set with
    #Ntest trajectories. The original trajectories contain 800 time steps but we only use 400 of them, we thus take
    #every second point
    Xtrain = X[0:Ntrain, 0:800:2]
    Xval = X[Ntrain:Ntrain+Nval, 0:800:2]
    Xtest = X[Ntrain+Nval:Ntrain+Nval+Ntest, 0:800:2]

    #extract the corresponding labels for the training, validation and test sets.
    Ytrain = Y[0:Ntrain, :]
    Yval = Y[Ntrain:Ntrain+Nval, :]
    Ytest = Y[Ntrain+Nval:Ntrain+Nval+Ntest, :]

    #calculating the Fourier coefficients for each subset.
    XtrainF = np.fft.fft(Xtrain)
    XvalF = np.fft.fft(Xval)
    XtestF = np.fft.fft(Xtest)

    #Prepare to split the Fourier coefficients into their real and imaginary components. Each complex number will 
    #occupy two columns: one for the real part and one for the imaginary part. Therefore, we create new arrays that 
    #have twice the number of columns. 
    xtrain = np.zeros((XtrainF.shape[0], 2*XtrainF.shape[1]))
    xval = np.zeros((XvalF.shape[0], 2*XvalF.shape[1]))
    xtest = np.zeros((XtestF.shape[0], 2*XtestF.shape[1]))

    #For each Fourier coefficient in the training set, split into real and imaginary parts. These parts are then
    #stored alternately (even indices for real, odd indices for imaginary).
    for i in range(XtrainF.shape[0]):
        for j in range(XtrainF.shape[1]):
            xtrain[i, 2*j] = XtrainF[i,j].real
            xtrain[i, 2*j + 1] = XtrainF[i,j].imag

    #Do the same for the test set, splitting the Fourier coefficients into their real and imaginary parts.
    for i in range(XtestF.shape[0]):
        for j in range(XtestF.shape[1]):
            xtest[i, 2*j] = XtestF[i,j].real
            xtest[i, 2*j + 1] = XtestF[i,j].imag

    #Similarly, split the Fourier coefficients for the validation set.
    for i in range(XvalF.shape[0]):
        for j in range(XvalF.shape[1]):
            xval[i, 2*j] = XvalF[i,j].real
            xval[i, 2*j + 1] = XvalF[i,j].imag
        
    #concatenating the original index from X as a new column to keep track of the trajectories. This index column
    #allows you to track each trajectory after operations like filtering and shuffling. 
    xtrain = np.concatenate((xtrain, X[0:Ntrain,-1].reshape(xtrain.shape[0], 1)), axis=1)
    xval = np.concatenate((xval, X[Ntrain:Ntrain+Nval,-1].reshape(xval.shape[0], 1)), axis=1)
    xtest = np.concatenate((xtest, X[Ntrain+Nval:Ntrain+Nval+Ntest,-1].reshape(xtest.shape[0], 1)), axis=1)

    #Return the transformed training, validation and test sets along with their corresponding labels
    return(xtrain, xval, xtest, Ytrain, Yval, Ytest)

In [ ]:
#Function to calculate the R-squared metric. This function takes the true and predicted values, and calculates the 
#R-squared value
def r_square(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))
    ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return(1 - ss_res/ss_tot)

In [ ]:
#define a HyperModel class for the Keras Tuner to optimise the architecture and hyperparameters of the model.
class HyperModel(kt.HyperModel):

    #function to build the model with hyperparameter tuning
    def build(self, hp):
        #create a sequential model
        model = tf.keras.Sequential()

        #Tune the number of neurons in the first dense layer between 32 and 512, with a step of 32. 'units_0' is the 
        #hyperparameter name, and it will be varied during tuning.
        model.add(tf.keras.layers.Dense(
            units=hp.Int('units_0', min_value = 32, max_value = 512, step=32),
            input_dim = (xtrainf.shape[1]-1),
            activation='relu'))
        
        #Tune the number of additional hidden layers between 0 and 10. For each layer, tune the number of neurons
        #between 32 and 512.
        for i in range(hp.Int('layers', 0, 10)):
            model.add(tf.keras.layers.Dense(
                units=hp.Int('units_' + str(i + 1), min_value=32, max_value=512, step=32),
                activation='relu'))

        #Add the output layer with 3 neurons (corresponding to the three peak positions in the regression task). 
        #Use the linear activation function for regression outputs
        model.add(tf.keras.layers.Dense(3,
                activation='linear'))


        #Tune the learning rate of the Adam optimiser, choosing from [0.01, 0,001, 0.0001, 0.00001, 0.000001]
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5, 1e-6])
        
        #Compile the model with the Adam optimiser, mean squared error loss, and r-squared metric
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = hp_learning_rate),
                      loss="mean_squared_error",
                      metrics=[r_square])
        
        #return the constructed model
        return(model)
    
    #function to fit the model, allowing the batch size to be tuned as well
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            #Tune the batch size by selecting from [16, 32, 61, half of the training set, or the full training set]
            batch_size=hp.Choice("batch_size", [16, 32, 64, int(xtrainf.shape[0]/2), xtrainf.shape[0]]),
            **kwargs,
        )

#instantiate a bayesian optimisation tuner
tuner = kt.BayesianOptimization(HyperModel(), #pass the HyperModel class
                     objective='val_loss', #Objective is to minimise the validation loss
                     max_trials = 100, #Perform up to 100 trials to explore different hyperparameter combinations
                     project_name='hp_optimisation_RCregression_threeRCsnu') #project name 

In [ ]:
#Retrieve the best hyperparameters from the search process
best_hps=tuner.get_best_hyperparameters(num_trials=100)[0] #Get the top hyperparameter combination from 100 trials

#print statements to display the optimal hyperparameters
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units_0')}.""")

#print the optimal number of hidden layers
print(f""" The optimal number of hidden layers is {best_hps.get('layers')}""")

#loop through and print the optimal number of units for each hidden layer
for i in range(best_hps.get('layers')):
  print(f""" The optimal number of units in layer {i + 1} is {best_hps.get('units_' + str(i + 1))}""")

#print the optimal learning rate and batch size
print(f"""the optimal learning rate for the optimizer is {best_hps.get('learning_rate')} and the optimal batch size is {best_hps.get('batch_size')}""")

In [ ]:
#define a range of epsilon values from 0 to 0.45 with a step of 0.05
epsilon = np.arange(0, 0.45+0.05, 0.05)

#initialise arrays to store the r_square and loss metric for the training, validation and test sets. These metrics
#will be recorded for each value of epsilon

#arrays to store the training r-squared and loss
trainingr2 = np.zeros(len(epsilon)) #r_squared for overall training set
trainingloss = np.zeros(len(epsilon)) #loss for overall training set

#arrays to store the validation r-squared and loss
valr2 = np.zeros(len(epsilon)) #r-squared for the overall validation set
valloss = np.zeros(len(epsilon)) #loss for the overall validation set

#arrays to store the test r-squared and loss
testr2 = np.zeros(len(epsilon)) #r-squared for overall test set
testloss = np.zeros(len(epsilon)) #loss for the overall test set

In [ ]:
#iterate over each epsilon value
for i in tqdm(range(len(epsilon))):
    #Initialise lists to store the filtered training data and corresponding $|nu$ values
    Xtrainx3_filtered = []
    nus3 = []

    #filter trajectories based on the epsilon criteria
    for j in range(xtrainx3_index.shape[0]): 
        #check if the absolute differences between $|nu$ values are greater than or equal to the current epsilon
        if np.abs(nu3[j,0] - nu3[j,1]) >= epsilon[i] and np.abs(nu3[j,0] - nu3[j,2]) >= epsilon[i] and np.abs(nu3[j,1]-nu3[j,2]) >= epsilon[i]:
            #append the filtered trajectory and corresponding $|nu$ values
            Xtrainx3_filtered.append(xtrainx3_index[[j],:])
            nus3.append(nu3[[j],:])

    #concatenate the filtered trajectories and $|nu$s into arrays
    xtrainx3_filtered_arr = np.concatenate(Xtrainx3_filtered)
    nus3_arr = np.concatenate(nus3)
    
    #sort each row of nus3_arr for consistency
    for k in range(nus3_arr.shape[0]):
        nus3_arr[k,:] = nus3_arr[k, nus3_arr[k,:].argsort()]
        
    #create an array of indices for shuffling
    indices = np.arange(xtrainx3_filtered_arr.shape[0])
    indices_shuffle = np.random.permutation(indices)
    
    #shuffling the filtered training data
    xtrain = xtrainx3_filtered_arr[indices_shuffle]
    ytrain = nus3_arr[indices_shuffle]
    
    #Defining sizes for training, validation and test sets (80-10-10 split)
    Ntrain = xtrainx3_filtered_arr.shape[0] - 2*int(xtrainx3_filtered_arr.shape[0]*0.1)
    Nval = int(xtrainx3_filtered_arr.shape[0]*0.1)
    Ntest = int(xtrainx3_filtered_arr.shape[0]*0.1)
    
    #scale the labels using MinMaxScaler for normalisation
    scaler = MinMaxScaler()
    ytrain_scaled = scaler.fit_transform(ytrain)
    
    #generate training, validation and test sets using the defined function
    xtrainf, xval, xtest, ytrainf, yval, ytest = fouriertrainvaltest(xtrain, ytrain_scaled, Ntrain, Nval, Ntest)

    #Build the model with the best hyperparameters
    model = HyperModel().build(best_hps)
    
    #Fit the model on the training data with validation
    history = model.fit(xtrainf[:,:-1], ytrainf, epochs = 1000, validation_data = (xval[:,:-1], yval), batch_size = best_hps.get('batch_size'), verbose=0)
    
    #save the trained model weights
    model.save("Weights/training_RCregression_threepeaksnu_eilson{0}.weights.h5".format(epsilon[i]))
    
    #evaluate the model on the training, validation and test sets and store the loss and r-squared metrics
    trainingloss[i], trainingr2[i] = model.evaluate(xtrainf[:,:-1], ytrainf)
    valloss[i], valr2[i] = model.evaluate(xval[:,:-1], yval)
    testloss[i], testr2[i] = model.evaluate(xtest[:,:-1], ytest)
    
    #save the loss and r-square metrics to csv files
    np.savetxt('traininglossvepsilon.csv', trainingloss, delimiter=',')
    np.savetxt('trainingr2vepsilon.csv', trainingr2, delimiter=',')
    np.savetxt('vallossvepsilon.csv', valloss, delimiter=',')
    np.savetxt('valr2vepsilon.csv', valr2, delimiter=',')
    np.savetxt('testlossvepsilon.csv', testloss, delimiter=',')
    np.savetxt('testr2vepsilon.csv', testr2, delimiter=',')
    
    #Make predictions for the training validation, and test sets
    predictions_train = scaler.inverse_transform(model.predict(xtrainf[:,:-1]))
    predictions_val = scaler.inverse_transform(model.predict(xval[:,:-1]))
    predictions_test = scaler.inverse_transform(model.predict(xtest[:,:-1]))
    
    #save predictions alongside the original index for training, validation and test sets
    np.savetxt('Predictions/predictionstrain_epsilon{0}.csv'.format(epsilon[i]), np.concatenate((predictions_train, xtrainf[:,[-1]]), axis=1), delimiter=',')
    np.savetxt('Predictions/predictionsval_epsilon{0}.csv'.format(epsilon[i]), np.concatenate((predictions_val, xval[:,[-1]]), axis=1), delimiter=',')
    np.savetxt('Predictions/predictionstest_epsilon{0}.csv'.format(epsilon[i]), np.concatenate((predictions_test, xtest[:,[-1]]), axis=1), delimiter=',')